In [1]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Lasso, LinearRegression, LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
import datetime
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
import math
from numpy.linalg import svd

from sklearn.ensemble import RandomForestClassifier

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori


### Loading the initial dataset

In [2]:
path = os.getcwd() + '/../dataset/anonymized_dataset_for_ADM2017/'
print(path + 'student_log_{}.csv'.format(1))
log1 = pd.read_csv(path + 'student_log_{}.csv'.format(1))
initial_col_order = [col.lower() for col in list(log1.columns)]

data = pd.DataFrame()
for i in range(1, 10):
    student_log = pd.read_csv(path + 'student_log_{}.csv'.format(i))
    student_log.columns = [col.lower() for col in student_log]
    data = pd.concat([data, student_log], ignore_index=True)

# reorder everything with the original order, where student id is in the first column
data = data[initial_col_order]
data.shape
data.head()

/Users/yeldosbalgabekov/Desktop/Books and Schools/UoSouthampton/Data Mining/DataMining/Yeldos_Richa/../dataset/anonymized_dataset_for_ADM2017/student_log_1.csv


/Users/yeldosbalgabekov/anaconda2/envs/py35/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (74,75) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,itest_id,sy assistments usage,aveknow,avecarelessness,avecorrect,numactions,averesbored,averesengcon,averesconf,averesfrust,...,confidence(off task),confidence(gaming),res_bored,res_concentrating,res_confused,res_frustrated,res_offtask,res_gaming,ln-1,ln
0,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.838710,0.008522,0.376427,0.320317,0.000000,0.0,0.785585,0.000264,0.13,0.0611904
1,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.600000,0.047821,0.156027,0.995053,0.887452,0.0,0.468252,0.001483,0.0611904,0.21351
2,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.600000,0.047821,0.156027,0.995053,0.887452,0.0,0.468252,0.001483,0.116,0.0333058
3,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.204082,0.343996,0.156027,0.744520,0.000000,0.0,0.108417,0.010665,0.116,0.0333058
4,8,2004-2005,0.352416,0.183276,0.483902,1056,0.208389,0.679126,0.115905,0.112408,...,0.204082,0.343996,0.156027,0.744520,0.000000,0.0,0.108417,0.010665,0.0333058,0.118386


### Transforming the initial dataset and loading Training Set

In [3]:
def to_float(x):
    try:
        if x.isdigit():
            return float(x)
    except:
        if x == '.':
            return 0
        return x

def series_types(series):
    types = []
    for i in set(series):
        if not type(i) in types:
            types.append(type(i))
    return(types)


data['sy assistments usage'] = data['sy assistments usage'].astype('category')
data['skill'] = data['skill'].astype('category')
data['problemtype'] = data['problemtype'].astype('category')
data['ln-1'] = data['ln-1'].apply(lambda x: to_float(x))
data['ln'] = data['ln'].apply(lambda x: to_float(x))
# data.memory_usage(deep=True)
# data.info(memory_usage='deep')

training = pd.read_csv(path + 'training_label.csv').drop(['AveCorrect'], axis=1)
training.columns = [col.lower() for col in training.columns]
training.shape

# merged data set with isSTEM
df = data.merge(training, on='itest_id', how="left")
#df.info()


### Filtering for the actions of the "training" students

In [4]:
actions = df[df.isstem.notnull()]
actions['skill'] = actions.skill.astype('category')
print("training set size: ", training.shape[0], "actions for # of students: ", len(actions.itest_id.unique()))
print("sample size useless for training: ", training.shape[0] - len(actions.itest_id.unique()))

# merged training data set
stem = actions[actions.isstem == 1]
nonstem = actions[actions.isstem == 0]

actions.head()

training set size:  514 actions for # of students:  326
sample size useless for training:  188


/Users/yeldosbalgabekov/anaconda2/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,itest_id,sy assistments usage,aveknow,avecarelessness,avecorrect,numactions,averesbored,averesengcon,averesconf,averesfrust,...,res_concentrating,res_confused,res_frustrated,res_offtask,res_gaming,ln-1,ln,schoolid,mcas,isstem
1056,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.262675,0.060808,0.000000,0.889219,0.005797,0.271000,0.077899,2.0,34.0,0.0
1057,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.696486,0.000000,0.009561,0.108417,0.001483,0.077899,0.225856,2.0,34.0,0.0
1058,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.580763,0.000000,0.009561,0.108417,0.003940,0.225856,0.483008,2.0,34.0,0.0
1059,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.580763,0.000000,0.009561,0.108417,0.003940,0.483008,0.745290,2.0,34.0,0.0
1060,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.898073,0.000000,0.009561,0.468252,0.001483,0.745290,0.900994,2.0,34.0,0.0


### Testing the cases
Analysing other features

In [5]:
actions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 177960 entries, 1056 to 639897
Data columns (total 79 columns):
itest_id                                  177960 non-null int64
sy assistments usage                      177960 non-null category
aveknow                                   177960 non-null float64
avecarelessness                           177960 non-null float64
avecorrect                                177960 non-null float64
numactions                                177960 non-null int64
averesbored                               177960 non-null float64
averesengcon                              177960 non-null float64
averesconf                                177960 non-null float64
averesfrust                               177960 non-null float64
averesofftask                             177960 non-null float64
averesgaming                              177960 non-null float64
actionid                                  177960 non-null int64
skill                            

In [6]:
actions.head()

,itest_id,sy assistments usage,aveknow,avecarelessness,avecorrect,numactions,averesbored,averesengcon,averesconf,averesfrust,...,res_concentrating,res_confused,res_frustrated,res_offtask,res_gaming,ln-1,ln,schoolid,mcas,isstem
1056,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.262675,0.060808,0.000000,0.889219,0.005797,0.271000,0.077899,2.0,34.0,0.0
1057,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.696486,0.000000,0.009561,0.108417,0.001483,0.077899,0.225856,2.0,34.0,0.0
1058,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.580763,0.000000,0.009561,0.108417,0.003940,0.225856,0.483008,2.0,34.0,0.0
1059,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.580763,0.000000,0.009561,0.108417,0.003940,0.483008,0.745290,2.0,34.0,0.0
1060,35,2004-2005,0.255164,0.158848,0.379658,993,0.222796,0.650079,0.069987,0.164347,...,0.898073,0.000000,0.009561,0.468252,0.001483,0.745290,0.900994,2.0,34.0,0.0


In [7]:
def threshold5(row):
    if row['total'] < 5:
        row['accuracy'] = 0
        return row
    row['accuracy'] = row['correct'] / row['total']
    return row

cols = ['itest_id', 'skill', 'correct']
truncated = actions[cols]
mapped_skills = dict(zip(truncated['skill'].astype('category').cat.codes.unique(), truncated['skill'].unique()))
mapped_skills
truncated['skill'] = truncated['skill'].astype('category').cat.codes
tmp = truncated
truncated.head()
truncated = truncated.groupby(['itest_id','skill'], as_index=False).count()
truncated.columns = ['itest_id', 'skill', 'total']
truncated['correct'] = tmp.groupby(['itest_id','skill'], as_index=False).sum().iloc[:, -1]
truncated = truncated.apply(lambda row: threshold5(row), axis=1)
truncated = truncated[['itest_id', 'skill', 'accuracy']]
truncated.head()

/Users/yeldosbalgabekov/anaconda2/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,itest_id,skill,accuracy
0,35.0,1.0,0.684211
1,35.0,3.0,0.000000
2,35.0,7.0,0.200000
3,35.0,13.0,0.315789
4,35.0,16.0,0.000000


In [85]:
top_no = 20
def get_top5(row, top_no):
    sorted_row = row[1:-1].sort_values()
    return list(sorted_row.index[-top_no:])
pivoted = truncated.pivot(index='itest_id', columns='skill', values='accuracy')
pivoted = pivoted.fillna(0)
pivoted = pd.DataFrame(pivoted.to_records())
tmp = actions[['itest_id', 'isstem']].drop_duplicates()
pivoted = pivoted.merge(tmp, how="left", on="itest_id")
# pivoted = pivoted[pivoted.isstem==1]
topname = 'top %s' % top_no
pivoted[topname] = pivoted.apply(lambda row: get_top5(row, top_no), axis=1)
pivoted = pivoted[['itest_id', topname, 'isstem']]
pivoted.head()

,itest_id,top 20,isstem
0,35,"[48.0, 60.0, 30.0, 55.0, 32.0, 27.0, 88.0, 49....",0.0
1,77,"[31.0, 38.0, 21.0, 74.0, 93.0, 13.0, 92.0, 73....",0.0
2,126,"[91.0, 70.0, 95.0, 77.0, 30.0, 65.0, 28.0, 42....",0.0
3,205,"[77.0, 34.0, 18.0, 65.0, 32.0, 30.0, 70.0, 29....",0.0
4,283,"[43.0, 55.0, 49.0, 67.0, 28.0, 71.0, 73.0, 70....",0.0


In [201]:
from mlxtend.frequent_patterns import association_rules

def add_label(row):
    el = 'stem' if row.isstem==1 else 'non_stem'
    res = list(row[topname])
    res.append(el)
    return res

storage = []
for i in range(100):
    feed = pivoted
    feed['with_label'] = pivoted.apply(lambda row: add_label(row), axis=1)

    g = feed.groupby('isstem', as_index=False)

    feed = pd.DataFrame(g.apply(lambda x: x.sample(g.size()
                                                   .min())
                                                   .reset_index(drop=True)))

    feed = list(feed['with_label'])

    te = TransactionEncoder()
    te_ary = te.fit(feed).transform(feed)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True, max_len=2)

    assres = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6)

    storage.append(np.array(assres[(assres.consequents == frozenset({'non_stem'})) | (assres.consequents == frozenset({'stem'}))])[:,:2])

In [202]:
preceds = []
conseqs = []
for session in storage:
    for row in session:
        preceds.append(list(row[0])[0])
        conseqs.append(list(row[1])[0])
counter = {}
for idx in range(len(preceds)):
    p, c = preceds[idx], conseqs[idx]
    if p in counter:
        if c == 'stem' and 'stem' in counter[p]:
            counter[p]['stem'] += 1
        else:
            counter[p]['stem'] = 1
        if c == 'non_stem' and 'non_stem' in counter[p]:
            counter[p]['non_stem'] += 1
        else:
            counter[p]['non_stem'] = 1
    else:
        if c == 'stem':
            counter[p] = {'stem': 1}
        else:
            counter[p] = {'non_stem': 1}

In [214]:
res = []
for el, chances in counter.items():
    if len(chances) != 2:
        choice = list(chances.keys())[0]
        if chances[choice] > 4:
            res.append((el, choice, chances[choice]))
    else:
        if chances['stem'] > chances['non_stem']:
            prop = chances['stem'] / chances['non_stem']
            choice = 'stem'
        else:
            prop = chances['non_stem'] / chances['stem']
            choice = 'non_stem'
        if prop > 4:
            res.append((el, choice, chances[choice]))

In [215]:
res

[('40.0', 'non_stem', 43),
 ('71.0', 'stem', 58),
 ('26.0', 'stem', 21),
 ('18.0', 'non_stem', 8),
 ('99.0', 'stem', 11),
 ('35.0', 'non_stem', 5)]

In [217]:
positives = []
negatives = []
for sk, ch, _ in res:
    if ch == 'stem':
        positives.append(sk)
    else:
        negatives.append(sk)
positives, negatives

(['71.0', '26.0', '99.0'], ['40.0', '18.0', '35.0'])

In [226]:
def ass_f_p(row, st):
    if st in row:
        return 1
    return 0

def ass_f_n(row, st):
    if st in row:
        return -1
    return 0

pre_fin = pivoted
for st in positives:
    pre_fin['ass_%s'%st] = pre_fin['top 20'].apply(lambda row: ass_f_p(row, st))
    
for st in negatives:
    pre_fin['ass_%s'%st] = pre_fin['top 20'].apply(lambda row: ass_f_n(row, st))

cols = ['itest_id'] + ['ass_%s'%st for st in positives] + ['ass_%s'%st for st in negatives]
pre_fin = pre_fin[cols]
pre_fin.head()

,itest_id,ass_71.0,ass_26.0,ass_99.0,ass_40.0,ass_18.0,ass_35.0
0,35,1,0,0,0,0,0
1,77,0,1,0,0,0,0
2,126,0,1,0,0,-1,0
3,205,0,0,0,0,-1,0
4,283,1,0,0,0,-1,0


### For the team

In [208]:
pivoted2 = truncated.pivot(index='itest_id', columns='skill', values='accuracy')
pivoted2 = pivoted2.fillna(0)
pivoted2 = pd.DataFrame(pivoted2.to_records())
pivoted2 = pivoted2

storage = []
for idx, table in pivoted2.iterrows():
    sorted_row = list(table)[1:]
    sorted_row.sort()
    threshold = sorted_row[-top_no]
    lst = [1 if (item > threshold and idx != 0) else 0 for idx, item in enumerate(list(table)[1:])]
    res = [list(table)[0]]
    res.extend(lst)
    storage.append(res)
storage = pd.DataFrame(np.array(storage))
storage = storage.rename(columns={0:'itest_id'})
storage.head()

,itest_id,1,2,3,4,5,6,7,8,9,...,84,85,86,87,88,89,90,91,92,93
0,35.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,77.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,126.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
3,205.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,283.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
storage.to_csv('../Features/just_top_10.csv')

In [237]:
to_csv = storage.merge(pre_fin, on="itest_id", how="left")
to_csv.to_csv('../Features/top_ass.csv')